# Figure panel 1 

In [3]:
options(warn=-1)

In [4]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # Data 
            library(tidyverse), 
            library(data.table), 

            # miloR
            library(miloR), 
            library(ggbeeswarm), 

            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(cowplot), 
            library(ComplexHeatmap), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [5]:
# Configure reticulate 
# use_condaenv(condaenv="p.3.10.16-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
# py_config()

In [6]:
random_seed <- 42
set.seed(random_seed)

In [7]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [8]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=4)) # From project global source()

In [9]:
# Helper functions
source("bin/so_pl.R")

source("bin/dea_pp.R")
source("bin/dea_pl.R")

source("bin/gsea_pp.R")
source("bin/gsea_pl.R")

source("bin/dp_feature.R")
source("bin/grn_pl.R")

# Import data 

In [10]:
so <- readRDS("data/scRNAseq/object/pp_0.rds")
so <- NormalizeData(so)

Normalizing layer: counts



In [11]:
so$sample_group <- factor(so$sample_group, levels=c("Bl6_NaCl_D6", "Bl6_CpG_D6"))

# Compute percentage primitive and definitive hemoglobin 

In [12]:
so <- PercentageFeatureSet(
    so,
    features = c("Hba-x", "Hbb-y", "Hbb-bh1"),
    col.name="pct_counts_primitive_HB", 
    assay = "RNA"
)

In [13]:
so <- PercentageFeatureSet(
    so,
    features = c("Hba-a1", "Hba-a2", "Hbb-bt", "Hbb-bs"),
    col.name="pct_counts_definitive_HB", 
    assay = "RNA"
)

# UMAP leiden celltype low

In [14]:
dp_1 <- dplot(so, group_by="celltype_low", alpha=1.0, pt_size=0.25, shuffle=TRUE, size_select=4, label=TRUE, label_box=TRUE, label_size=6) + 

    scale_color_manual(values=color$celltype_low) + 
    scale_fill_manual(values=color$celltype_low) + 
    guides(color=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0, keyheight=0.25, default.unit="cm")) + 
    theme(
    
        legend.text.align=0
    
    ) 

Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


In [15]:
pdf("result/figures/1_scRNAseq/panel_1/umap_celltype_low.pdf", width=3.5, height=2.5)

dp_1

dev.off()

pdf 
  2

# UMAP FACS

In [16]:
so$facs <- factor(so$facs, levels=c("Progenitor", "Myeloid"))

In [17]:
dp_1 <- dplot(so, group_by="facs", alpha=1.0, pt_size=0.25, shuffle=FALSE, size_select=4, label=FALSE, label_box=FALSE, label_size=6) + 

    scale_color_manual(values=color$facs) + 
    scale_fill_manual(values=color$facs) + 
    guides(color=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0, keyheight=0.25, default.unit="cm")) + 
    theme(
    
        legend.text.align=0
    
    ) 

In [18]:
pdf("result/figures/1_scRNAseq/panel_1/umap_facs.pdf", width=3.5, height=2.5)

gridExtra::grid.arrange(
    
    dp_1 %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm"))

)

dev.off()

pdf 
  2

# UMAP Ctr vs CpG

In [19]:
so_tmp <- so

In [20]:
color_tmp <- color$sample_group
color_tmp["Bl6_CpG_D6"] <- "#7F7F7F"

In [21]:
data <- cbind(so_tmp[["umap"]]@cell.embeddings, so_tmp@meta.data)
data <- data[data$sample_group=="Bl6_NaCl_D6", ]

In [22]:
dp_1 <- dplot(so_tmp, group_by="sample_group", alpha=1, pt_size=0.25, shuffle=FALSE, size_select=4, order=c("Bl6_NaCl_D6", "Bl6_CpG_D6")) + 
    scale_color_manual(values=color_tmp) + 
    guides(color=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0, keyheight=0.25, default.unit="cm")) + 
    stat_density_2d(data=data, aes(x=UMAP_1, y=UMAP_2), color="black", size=0.2, adjust=0.5, n=100, contour=TRUE) + 
    theme(legend.text.align=0) 

In [23]:
pdf("result/figures/1_scRNAseq/panel_1/umap_wt_nacl_d6.pdf", width=3.5, height=2.5)

dp_1
dev.off()

pdf 
  2

In [24]:
so_tmp <- so

In [25]:
color_tmp <- color$sample_group
color_tmp["Bl6_NaCl_D6"] <- "#7F7F7F"

In [26]:
data <- cbind(so_tmp[["umap"]]@cell.embeddings, so_tmp@meta.data)
data <- data[data$sample_group=="Bl6_CpG_D6", ]

In [27]:
dp_1 <- dplot(so_tmp, group_by="sample_group", alpha=1, pt_size=0.25, shuffle=FALSE, size_select=4, order=c("Bl6_CpG_D6", "Bl6_NaCl_D6")) + 
    scale_color_manual(values=color_tmp) + 
    guides(color=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0, keyheight=0.25, default.unit="cm")) + 
    stat_density_2d(data=data, aes(x=UMAP_1, y=UMAP_2), color="black", size=0.2, adjust=0.5, n=100, contour=TRUE) + 
    theme(legend.text.align=0) 

In [28]:
pdf("result/figures/1_scRNAseq/panel_1/umap_wt_cpg_d6.pdf", width=3.5, height=2.5)

dp_1

dev.off()

pdf 
  2

# Barplot FACS ratio per celltype

In [29]:
bp_1 <- ggplot(so@meta.data %>% dplyr::mutate(celltype_low=factor(celltype_low, levels=rev(levels(celltype_low)))), aes(x=celltype_low, fill=facs)) + 
    geom_bar(position="fill", width=0.8, color="black", size=0.1) + 
    scale_fill_manual(values=color$facs) + 
    ggtitle("") + xlab("") + ylab("Celltype [ratio]") + 
    coord_flip() +
    theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [30]:
pdf("result/figures/1_scRNAseq/panel_1/bp_facs_celltype_low.pdf", width=2.5, height=2.5)

bp_1

dev.off()

pdf 
  2

# Barplot celltype ratio  

In [31]:
so$celltype_high <- ifelse(so$celltype_low %in% c("MEP", "Proerythroblast", "Erythroblast"), "E", ifelse(so$celltype_low %in% c("MegP", "GMP", "NeuP", "BasoP", "MastP"), "P", "M"))

In [32]:
so$sample_group_rep <- factor(so$sample_group_rep, levels=c("Bl6_NaCl_D6_Rep1", "Bl6_NaCl_D6_Rep2", "Bl6_CpG_D6_Rep1",  "Bl6_CpG_D6_Rep2" ))

In [33]:
bp_1 <- ggplot(so@meta.data[so$celltype_high=="E", ], aes(x=sample_group_rep, fill=celltype_low)) + 
    geom_bar(position="fill", width=0.8, color="black", size=0.1) + 
    scale_fill_manual(values=color$celltype_low) + 
    ggtitle("") + xlab("") + ylab("Celltype [ratio]") + 
    facet_wrap(~celltype_high, scales="free", ncol=1) + 
    scale_x_discrete(labels=c("Rep1", "Rep2", "Rep1", "Rep2")) + 
    theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

bp_2 <- ggplot(so@meta.data[so$celltype_high=="P", ], aes(x=sample_group_rep, fill=celltype_low)) + 
    geom_bar(position="fill", width=0.8, color="black", size=0.1) + 
    scale_fill_manual(values=color$celltype_low) + 
    ggtitle("") + xlab("") + ylab("Celltype [ratio]") + 
    facet_wrap(~celltype_high, scales="free", ncol=1) + 
    scale_x_discrete(labels=c("Rep1", "Rep2", "Rep1", "Rep2")) +
    theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

bp_3 <- ggplot(so@meta.data[so$celltype_high=="M", ], aes(x=sample_group_rep, fill=celltype_low)) + 
    geom_bar(position="fill", width=0.8, color="black", size=0.1) + 
    scale_fill_manual(values=color$celltype_low) + 
    ggtitle("") + xlab("") + ylab("Celltype [ratio]") + 
    facet_wrap(~celltype_high, scales="free", ncol=1) + 
    scale_x_discrete(labels=c("Rep1", "Rep2", "Rep1", "Rep2")) +
    theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [34]:
pdf("result/figures/1_scRNAseq/panel_1/bp_celltype_low.pdf", width=2, height=4)

gridExtra::grid.arrange(

    bp_1 %>% egg::set_panel_size(., width=unit(4*0.3, "cm"), height=unit(2.0, "cm")),
    bp_2 %>% egg::set_panel_size(., width=unit(4*0.3, "cm"), height=unit(2.0, "cm")),
    bp_3 %>% egg::set_panel_size(., width=unit(4*0.3, "cm"), height=unit(2.0, "cm")), ncol=1
        
)

dev.off()

pdf 
  2

# Celltype marker genes

In [35]:
marker_genes=data.frame(

    celltype=c(
        
        "Prog.", "Prog.", "Prog.", "Prog.", "Prog.",

        "NeuP", 
               
        "Baso.", "Baso.", "Baso.", "Baso.", 
               
        "Mast", "Mast", 

        "MegP", "MegP", 
               
        "Erythroid", "Erythroid", "Erythroid", "Erythroid", "Erythroid", "Erythroid", "Erythroid", "Erythroid", "Erythroid", 
               
        "Mono.", "Mono.", "Mono.", "Mono.", 
               
        "RPM", "RPM", "RPM", 
               
        "cDC", "cDC", "cDC", "cDC", "cDC", "cDC", "cDC"
               
    ), 
    
    genes=c(
        
        "Hlf", "Cd34", "Kit", "Meis1", "Spi1",  

        "Elane", 
            
        "Cebpa", "Mcpt8", "Clec12a", "Itga2", 
            
        "Mcpt4", "Cma1", 

        "Pf4", "Itga2b",
            
        "Gata2", "Gata1", "Klf1", "Icam4", "Epor", "Sox6", "Tfrc", "Alas2", "Slc4a1",
            
        "Ly6c2", "Ccr2", "Cx3cr1", "Itgam",
            
        "Adgre1", "Spic", "Cd163",  
            
        "Sirpa", "Flt3", "Xcr1", "Cd8a", "Cd4", "Cd209a", "Ccr7"
    
    )
    
)

In [36]:
dp_1 <- dp_feature(so, marker_genes$genes, group_by="celltype_low", group_by_order=rev(levels(so$celltype_low)), split=marker_genes$celltype, split_order=unique(marker_genes$celltype), range_max=2.5) + 
    scale_y_discrete(labels=rev(levels(so$celltype_low))) + theme_global_set(4) + 

        theme(
            legend.position="bottom", 
            panel.spacing=unit(0.1, "lines"), 
            axis.text.x=element_text(angle=90, vjust=0.5, hjust=1, face="italic"),
            axis.text.y=element_text(angle=0, vjust=0.5, hjust=1), 
            strip.text.x=element_text(angle=90, vjust=0.5, hjust=0), 
            strip.text=element_text(margin=margin(0.1, 0.1, 0.1, 0.1, "lines")), 
            strip.background=element_rect(fill="transparent", color=NA), 
            legend.key.size=unit(0.25, "cm"), 
            legend.key.height=unit(0.25, "cm"), 
            legend.key.width=unit(0.25, "cm")
            
        )

`summarise()` has grouped output by 'celltype_low'. You can override using the
`.groups` argument.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


In [37]:
pdf("result/figures/1_scRNAseq/panel_1/dp_celltype_low_marker_genes.pdf", width=4.0, height=3.2)

dp_1

dev.off()

pdf 
  2

# Celltype abundance test with milo (NaCl D6 vs CpG D6 WT)

In [38]:
# Import Milo object
milo <- readRDS(paste0("data/scRNAseq/milo/milo_k30.rds"))
results <- readRDS("data/scRNAseq/milo/resl.rds")
result <- results[[2]][["res_da"]][[1]]

In [39]:
alpha <- 0.05
group_by <- "celltype_low"

In [40]:
# Beeswarm 
data <- suppressMessages(plotDAbeeswarm(result, group.by=group_by, alpha=alpha))

In [41]:
data <- data$data %>%
    mutate(is_signif=ifelse(SpatialFDR < alpha, 1, 0)) %>%
    mutate(logFC_color=ifelse(is_signif==1, logFC, NA)) %>%
    arrange(group_by) %>%
    mutate(Nhood=factor(Nhood, levels=unique(Nhood))) %>%
    mutate(pos_x=pos_x, pos_y=pos_y) 

In [42]:
y_breaks <- c(-floor(max(abs(data$logFC))), -floor(max(abs(data$logFC)))/2, 0, floor(max(abs(data$logFC)))/2, floor(max(abs(data$logFC))))
n_groups <- length(unique(data$group_by))

In [43]:
options(repr.plot.width=4, repr.plot.height=10)

dabp <- ggplot(data, aes(pos_x, pos_y, color=logFC_color)) +
    scale_color_gradient2(low=color$sample_group["Bl6_NaCl_D6"], high=color$sample_group["Bl6_CpG_D6"]) + 
    guides(color="none") +
    xlab("") + ylab("Log Fold Change") + ggtitle("Differential abundance") + 
    scale_x_discrete(labels=setNames(levels(data$group_by), seq(1 ,n_groups))) + 
    scale_y_continuous(breaks=y_breaks, labels=y_breaks) +
    geom_hline(yintercept=0, size=0.1) + 
    geom_point(size=0.25) + 
    stat_summary(aes(x=group_by, y=logFC), fun.y=median, geom="point", size=0.5, color="black") +
    coord_flip() +
    ylim(-max(abs(data$logFC)), max(abs(data$logFC))) + 
    theme(axis.text.y=element_text(vjust=0.5, hjust=1))

Scale for y is already present.
Adding another scale for y, which will replace the existing scale.


In [44]:
pdf("result/figures/1_scRNAseq/panel_1/dp_erythroid_milo.pdf", width=1.5, height=2.5)

dabp + theme_global_set(size_select=4)

dev.off()

pdf 
  2

# DEA 

# DEA overview 

In [45]:
adj_pval_thr <- 0.05
log2FC_thr  <- 0.25

In [46]:
contrasts_vec <- c("Bl6_CpG_D6", "Bl6_NaCl_D6")
dea_res_1 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [47]:
mat_1 <- lapply(dea_res_1, function(x) {x %>% dplyr::filter(p_val_adj <= adj_pval_thr & abs(avg_log2FC)>=log2FC_thr ) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t()

In [48]:
pdf("result/figures/1_scRNAseq/panel_1/hm_dea_count.pdf", width=1.5, height=3)

draw(dea_count_hm(mat_1, fontsize_select=2))

dev.off()

pdf 
  2

# Gene set enrichment analysis (GSEA)

## Gene Set M7

In [49]:
compute_gsea <- FALSE
celltype_select <- names(dea_res_1)

In [50]:
if(compute_gsea) {

    # Collapse gene set
    gene_set <- msigdbr::msigdbr(species="mouse", db_species="MM", category="M7", subcategory=NULL)
    gene_set <- gene_set[grep("_UP", gene_set$gs_name), ]
    gene_set$gs_name <- sapply(strsplit(gene_set$gs_name, "_"), function(x) {paste(x[max(1, length(x)-3):(length(x)-2)], collapse="_")})
    gene_set$gs_name <- gsub("CELL_|CD4_|CD8_|MONOCYTE_|MACROPHAGE_|TREG_|LANGERHANS_|NEUTROPHIL_|CDC1_|CDC2_|MIGDC_|ETAC_|GD_|ILC_|PDC_", "", gene_set$gs_name)
    gene_set <- gene_set %>% dplyr::select(gene_symbol, gs_name) %>% dplyr::distinct()

    gsea_res <- lapply(celltype_select, function(i) {
    
        gsea_res <- gsea(dea_res_1[[i]], gene_set=gene_set) %>% dplyr::mutate(col_split="Ctl vs D6", col_label=i)
    
        return(gsea_res)
        
    }
          )

    saveRDS(gsea_res, "result/gsea/scRNAseq/gsea_d6_vs_ctl.rds")
    
} else {

    gsea_res <- readRDS("result/gsea/scRNAseq/gsea_d6_vs_ctl.rds")
    
}

In [51]:
gsea_res <- do.call(rbind, gsea_res)

In [52]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05 & abs(NES)>=1) %>% dplyr::pull(pathway)

In [53]:
mat_1 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [54]:
mat_2 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [55]:
row_split <- read.csv("data/reference/immune_dictionary/immune_dictionary.csv") %>% tibble::column_to_rownames("gs_name")
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [56]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [57]:
row_split <- row_split[rownames(mat_1), ]

In [58]:
rownames(mat_1) <- row_split$norm
rownames(mat_2) <- row_split$norm

In [59]:
row_split <- row_split %>% dplyr::select(norm, family) %>% dplyr::rename(row_split_label=family) %>% dplyr::distinct()
rownames(row_split) <- row_split$norm

In [60]:
col_label[col_label=="cDC.mig."] <- "cDC mig."

In [61]:
pdf("result/figures/1_scRNAseq/panel_1/hm_gsea_d6_vs_ctl_m7.pdf", width=8, height=12)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=row_split, color_neg=color$sample_group[1], color_pos=color$sample_group[2], fontsize_select=2)

dev.off()

pdf 
  2

## genotype comparison - HM

In [62]:
compute_gsea <- FALSE
celltype_select <- names(dea_res_1)

In [63]:
if(compute_gsea) {

    gsea_res <- lapply(celltype_select, function(i) {
    
        gsea_res <- gsea(dea_res_1[[i]], category="MH", subcategory=NULL, gene_set=NULL) %>% dplyr::mutate(col_split="Ctl vs D6", col_label=i)
    
        return(gsea_res)
        
    }
          )

    saveRDS(gsea_res, "result/gsea/scRNAseq/gsea_d6_vs_ctl_mh.rds")
    
} else {

    gsea_res <- readRDS("result/gsea/scRNAseq/gsea_d6_vs_ctl_mh.rds")
    
}

In [64]:
gsea_res <- do.call(rbind, gsea_res)

In [65]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05 & abs(NES)>=1) %>% dplyr::pull(pathway)

In [66]:
gs_name_select <- c(
    
    "HALLMARK_E2F_TARGETS",
    "HALLMARK_MYC_TARGETS_V1",
    "HALLMARK_MYC_TARGETS_V2",
    "HALLMARK_HEME_METABOLISM",
    
    "HALLMARK_INTERFERON_ALPHA_RESPONSE",
    "HALLMARK_TNFA_SIGNALING_VIA_NFKB",
    
    "HALLMARK_OXIDATIVE_PHOSPHORYLATION"

) 

In [67]:
mat_1 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [68]:
mat_2 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [69]:
mat_1 <- mat_1[, c("col_label", "col_split", gs_name_select)]
mat_2 <- mat_2[, c("col_label", "col_split", gs_name_select)]

In [70]:
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [71]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [72]:
col_label[col_label=="cDC.mig."] <- "cDC mig."

In [73]:
pdf("result/figures/1_scRNAseq/panel_1/hm_gsea_d6_vs_ctl_mh.pdf", width=8, height=10)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=NULL, pathway_suffix="HALLMARK", color_neg=color$sample_group[1], color_pos=color$sample_group[2], fontsize_select=2)

dev.off()

pdf 
  2

## genotype comparison - Reactome

In [74]:
compute_gsea <- FALSE
celltype_select <- names(dea_res_1)

In [75]:
if(compute_gsea) {

    gsea_res <- lapply(celltype_select, function(i) {
    
        gsea_res <- gsea(dea_res_1[[i]], category="M2", subcategory="CP:REACTOME", gene_set=NULL) %>% dplyr::mutate(col_split="Ctl vs D6", col_label=i)
    
        return(gsea_res)
        
    }
          )

    saveRDS(gsea_res, "result/gsea/scRNAseq/gsea_d6_vs_ctl_m2_cp_reactome.rds")
    
} else {

    gsea_res <- readRDS("result/gsea/scRNAseq/gsea_d6_vs_ctl_m2_cp_reactome.rds")
    
}

In [76]:
gsea_res <- do.call(rbind, gsea_res)

In [77]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05 & abs(NES)>=1) %>% dplyr::pull(pathway)

In [78]:
mat_1 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [79]:
mat_2 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [80]:
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [81]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [82]:
col_label[col_label=="cDC.mig."] <- "cDC mig."

In [83]:
pdf("result/figures/1_scRNAseq/panel_1/hm_gsea_d6_vs_ctl_m2_cp_reactome.pdf", width=8, height=20)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=NULL, pathway_suffix="REACTOME", color_neg=color$sample_group[1], color_pos=color$sample_group[2], fontsize_select=2)

dev.off()

pdf 
  2

## genotype comparison - GO

In [84]:
compute_gsea <- FALSE
celltype_select <- names(dea_res_1)

In [85]:
if(compute_gsea) {

    gsea_res <- lapply(celltype_select, function(i) {
    
        gsea_res <- gsea_cp(dea_res_1[[i]]) %>% dplyr::mutate(col_split="Ctl vs D6", col_label=i)
    
        return(gsea_res)
        
    }
          )

    saveRDS(gsea_res, "result/gsea/scRNAseq/gsea_d6_vs_ctl_gobp.rds")
    
} else {

    gsea_res <- readRDS("result/gsea/scRNAseq/gsea_d6_vs_ctl_gobp.rds")
    
}

In [86]:
gsea_res <- do.call(rbind, gsea_res)

In [87]:
gs_name_select <- gsea_res %>% dplyr::filter(p.adjust<=0.05 & abs(NES)>=1) %>% dplyr::pull(ID)
gsea_res <- gsea_res %>% dplyr::filter(ID %in% gs_name_select)

In [88]:
gsea_res <- gsea_res[!gsea_res$Description %in% grep("T cell|B cell|CD4|CD8|lymphocyte|endothelial|epithelial", gsea_res$Description, value=TRUE), ]

In [89]:
if(compute_gsea) {

    gs_reduce <- gsea_cp_reduce(gsea_res)

    saveRDS(gs_reduce, "result/gsea/scRNAseq/gsea_reduce_d6_vs_ctl_gobp.rds")
    
} else {

    gs_reduce <- readRDS("result/gsea/scRNAseq/gsea_reduce_d6_vs_ctl_gobp.rds")
    
}

In [90]:
gsea_res <- gsea_res %>% dplyr::rename(pathway=Description, padj=p.adjust) %>% dplyr::filter(ID %in% gs_reduce$go)

In [91]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.1 & abs(NES)>=1) %>% dplyr::pull(ID)
gsea_res <- gsea_res %>% dplyr::filter(ID %in% gs_name_select)

In [92]:
mat_1 <- gsea_res %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [93]:
mat_2 <- gsea_res %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [94]:
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [95]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [96]:
row_split <- gs_reduce %>% dplyr::select(term, parentTerm) %>% dplyr::rename(row_split_label=parentTerm) %>% dplyr::distinct()
rownames(row_split) <- gs_reduce$term

In [97]:
col_label[col_label=="cDC.mig."] <- "cDC mig."

In [98]:
pdf("result/figures/1_scRNAseq/panel_1/hm_gsea_d6_vs_ctl_gobp.pdf", width=10, height=100)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=row_split, color_neg=color$sample_group[1], color_pos=color$sample_group[2], pathway_suffix="WP", fontsize_select=2)

dev.off()

pdf 
  2

## Volcano plot

In [163]:
contrasts_vec <- c("Bl6_CpG_D6", "Bl6_NaCl_D6")
dea_res_1 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [164]:
labels <- list(

    "RPM"=c(
        
        "Gdf15", "Spic", "Zeb2", "Vcam1", "Mertk", "Pparg", # RPM marker 
        "Cd44", "Lrp1", "Lgf1", # Residency
        "Slc40a1", "Hmox1", "Abca1", # Iron metabolism 
        "Cybb", "Ncf1", "Ctss", "Fcgr2", "Ctsl", "Ctsd", "Lgals3", "Marco", "Msr1", "Fcgr1", # Phagocytosis 
        "H2-Ab1", "H2-Eb1", "Cd74", 
        "Nfkbia", "Relb", # NFKB
        "Batf", "Batf3", 
        "Ifit3", "Ddx60", "Ifitm1", "Ifitm2", "Ifitm6", "Bst2", "Psmb10", "Parp9", "Ccl5", # Interferon signaling
        "Cxcl9", "Bcl2a1a", "Bcl2a1b", "Bcl2a1d" # Survival 
        
        
    
    ), 

    "intMo"=c(
        
        "Gdf15", "Spic", "Zeb2", "Vcam1", "Mertk", "Pparg", # RPM marker 
        "Cd44", "Lrp1", "Lgf1", # Residency
        "Slc40a1", "Hmox1", "Abca1", # Iron metabolism 
        "Cybb", "Ncf1", "Ctss", "Fcgr2", "Ctsl", "Ctsd", "Lgals3", "Marco", "Msr1", "Fcgr1", # Phagocytosis 
        "H2-Ab1", "H2-Eb1", "Cd74", 
        "Nfkbia", "Relb", # NFKB
        "Batf", "Batf3", 
        "Ifit3", "Ddx60", "Ifitm1", "Ifitm2", "Ifitm6", "Bst2", "Psmb10", "Parp9", "Ccl5", # Interferon signaling
        "Cxcl9", "Bcl2a1a", "Bcl2a1b", "Bcl2a1d" # Survival 
    
    ), 

    "cMo"=c(
        
        "Gdf15", "Spic", "Zeb2", "Vcam1", "Mertk", "Pparg", # RPM marker 
        "Cd44", "Lrp1", "Lgf1", # Residency
        "Slc40a1", "Hmox1", "Abca1", # Iron metabolism 
        "Cybb", "Ncf1", "Ctss", "Fcgr2", "Ctsl", "Ctsd", "Lgals3", "Marco", "Msr1", "Fcgr1", # Phagocytosis 
        "H2-Ab1", "H2-Eb1", "Cd74", 
        "Nfkbia", "Relb", # NFKB
        "Batf", "Batf3", 
        "Ifit3", "Ddx60", "Ifitm1", "Ifitm2", "Ifitm6", "Bst2", "Psmb10", "Parp9", "Ccl5", # Interferon signaling
        "Cxcl9", "Bcl2a1a", "Bcl2a1b", "Bcl2a1d" # Survival 
    
    )
    
)

In [165]:
options(repr.plot.width=6*6, repr.plot.height=6)

vp_1 <- lapply(names(dea_res_1), function(i) {

    if(i %in% names(labels)) {

        label <- labels[[i]]
        
    } else {

        label <- NULL
        
    }

    v_pl(dea_res_1[[i]], title=i, log2FC_thr=log2FC_thr , adj_pval_thr=adj_pval_thr, color_pos=color$sample_group[contrasts_vec[1]], color_neg=color$sample_group[contrasts_vec[2]], label=label, top_label=20, label_merge=FALSE, point_size=0.5, label_size=2, aspect_ratio=0.8, y_limit=3) + theme(legend.position="none") + theme_global_set(size_select=4)

}
              )

In [166]:
pdf("result/figures/1_scRNAseq/panel_1/v_plot_d6_vs_ctl.pdf", width=5*2, height=4*2)

plots <- lapply(vp_1, function(p) egg::set_panel_size(p, width=unit(3, "cm"), height=unit(3, "cm")))
do.call(gridExtra::grid.arrange, c(plots, ncol=5, nrow=4, newpage=FALSE))

dev.off()

pdf 
  2

## log2FC DEA heatmap of selected genes 

In [150]:
log2fc_thr <- 0.25
p_val_adj_thr <- 0.05
celltype_select <- c("cMo", "ncMo", "intMo", "RPM", "cDC1", "cDC2", "moDC", "cDC mig.", "Basophil")

In [151]:
genes_dea <- lapply(dea_res_1, function(x) {x %>% dplyr::filter(abs(avg_log2FC)>=log2fc_thr & p_val_adj<=p_val_adj_thr) %>% dplyr::pull(gene)})
genes_dea <- do.call("c", genes_dea) %>% unique()

In [152]:
row_genes <- list(
    
    "MHC I"=c("H2-K1", "H2-D1", "B2m"),
    # "MHC II"=c("H2-Ab1", "H2-Eb1", "Cd74"),
    "Antigen/Proteasome"=c("Psmb8", "Psmb9", "Psme1", "Psme2"),
    # "Phagosome/Autophagy"=c("Ctss", "Ctsb", "Lamp1", "Lamp2", "Atp6v0d2", "Map1lc3b", "Becn1", "Atg5"),
    "IFN signaling"=c("Ly6a", "Ifitm1", "Ifitm2", "Ifitm3", "Gbp2", "Gbp7", "Gbp8"),
    # "NFkB/MAPK"=c("Tlr9", "Myd88", "Irak4", "Irak1", "Traf6", "Map3k7", "Mapk14", "Mapk8", "Nfkb1", "Nfkb2", "Rela", "Relb", "Nfkbia"),
    # "AP1"=c("Fos", "Jun", "Junb", "Jund", "Fosb", "Fosl1", "Atf3", "Batf3"),
    # "Inflammasome"=c("Pycard", "Nlrp3", "Nlrp1b", "Il1b", "Il18"),
    # "Survival"=c("Cxcl9", "Havcr2", "Bcl2a1a", "Bcl2a1b", "Bcl2a1d"), 
    # "ROS/NOX2 complex"=c("Cybb", "Cyba", "Ncf1", "Ncf2", "Ncf4", "Rac2"),
    # "RPM iron/heme program"=c("Spic", "Slc40a1", "Hmox1", "Fth1", "Ftl1", "Hamp"), 
    # "Glycolysis/Lactate"=c("Slc2a1", "Hk2", "Pfkp", "Pfkfb3", "Aldoa", "Gapdh", "Pgk1", "Pkm", "Ldha", "Slc16a1"),
    "OXPHOS"=c("Ndufb8", "Uqcrb", "Cox5a", "Cox8a", "Atp5b", "Atp5c1")
    # "Pentose Phosphate"=c("G6pdx", "Pgls", "Rpe", "Rpia", "Tkt", "Taldo1"),
    # "Hexosamine Pathway"=c("Gfpt1", "Gfpt2", "Gnpda1", "Ogt", "Oga")

)

In [153]:
row_split <- setNames(unlist(row_genes, use.names = FALSE), rep(names(row_genes), sapply(row_genes, length)))
row_split[!row_split %in% rownames(so)]

named character(0)

In [154]:
row_split <- row_split[row_split %in% genes_dea]

In [155]:
mat_1 <- lapply(names(dea_res_1), function(i) {dea_res_1[[i]] %>% dplyr::mutate(col_name=i) %>% dplyr::select(avg_log2FC, col_name) %>% tibble::rownames_to_column("gene")})
mat_2 <- lapply(names(dea_res_1), function(i) {dea_res_1[[i]] %>% dplyr::mutate(col_name=i) %>% dplyr::select(p_val_adj, col_name) %>% tibble::rownames_to_column("gene")})

In [156]:
mat_1 <- do.call("rbind", mat_1)  %>% reshape2::dcast(., gene~col_name, value.var="avg_log2FC", fill=0) %>% tibble::column_to_rownames("gene")
mat_2 <- do.call("rbind", mat_2) %>% reshape2::dcast(., gene~col_name, value.var="p_val_adj", fill=1) %>% tibble::column_to_rownames("gene")

In [157]:
colnames(mat_1)[colnames(mat_1)=="cDC.mig."] <- "cDC mig."
colnames(mat_2)[colnames(mat_2)=="cDC.mig."] <- "cDC mig."

In [158]:
mat_1 <- mat_1[row_split, celltype_select]
mat_2 <- mat_2[row_split, celltype_select]

In [159]:
mat_1[is.na(mat_1)] <- 0
mat_2[is.na(mat_2)] <- 1

In [160]:
sig_bool_thr <- rowSums(mat_2<=0.05)>0

mat_1 <- mat_1[sig_bool_thr, ]
mat_2 <- mat_2[sig_bool_thr, ]
row_split <- row_split[sig_bool_thr]

In [161]:
pdf("result/figures/1_scRNAseq/panel_1/hm_dea_res_myeloid.pdf", width=3, height=10)
source("bin/dea_pl.R")
dea_res_hm(mat_1, mat_2, row_split=factor(names(row_split), levels=unique(names(row_split))), scale_width=1, color_min=color$sample_group[contrasts_vec[2]], color_max=color$sample_group[contrasts_vec[1]], cluster_rows=FALSE, cluster_columns=FALSE, fontsize_select=2)

dev.off()

pdf 
  2

# Expression UMAP

In [ ]:
so <- NormalizeData(so, normalization.method="LogNormalize", scale.factor=1e4)

In [ ]:
cell_id_thr <- so@reductions$umap@cell.embeddings %>% as.data.frame() %>% dplyr::filter(UMAP_1==min(UMAP_1) | UMAP_1==max(UMAP_1) | UMAP_2==min(UMAP_2) | UMAP_2==max(UMAP_2)) %>% rownames()

In [ ]:
so_tmp <- so[, (so$celltype_low %in% c("GMP", "NeuP", "BasoP", "Basophil", "MastP", "MegP", "MEP", "Proerythroblast", "Erythroblast")) | so$cell_id %in% cell_id_thr]

In [ ]:
pdf("result/figures/1_scRNAseq/panel_1/umap_marker_genes.pdf", width=10, height=10)

    gridExtra::grid.arrange(

        fplot(so_tmp, features="pct_counts_definitive_HB", assay="RNA", slot="data", pt_size=0.25, size_select=4, max_cutoff=30) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
        fplot(so_tmp, features="pct_counts_primitive_HB", assay="RNA", slot="data", pt_size=0.25, size_select=4, max_cutoff=30) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
        fplot(so_tmp, features="Cd34", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
        fplot(so_tmp, features="Meis1", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
        fplot(so_tmp, features="Gata1", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
        fplot(so_tmp, features="Gata2", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
        fplot(so_tmp, features="Klf1", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
        fplot(so_tmp, features="Pf4", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
        fplot(so_tmp, features="Sox6", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
        fplot(so_tmp, features="Elane", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
        fplot(so_tmp, features="Prss34", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
        fplot(so_tmp, features="Cma1", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
        fplot(so_tmp, features="Spi1", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
            
        fplot(so_tmp, restrict=c("sample_group", "Bl6_NaCl_D6"), features="Gzmb", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
        fplot(so_tmp, restrict=c("sample_group", "Bl6_CpG_D6"), features="Gzmb", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
            
        fplot(so_tmp, restrict=c("sample_group", "Bl6_NaCl_D6"), features="Hes1", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
        fplot(so_tmp, restrict=c("sample_group", "Bl6_CpG_D6"), features="Hes1", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")),
            
        fplot(so_tmp, restrict=c("sample_group", "Bl6_NaCl_D6"), features="Egr1", assay="RNA", slot="data", pt_size=0.25, size_select=4) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
        fplot(so_tmp, restrict=c("sample_group", "Bl6_CpG_D6"), features="Egr1", assay="RNA", slot="data", pt_size=0.25, size_select=4, max_set=5) %>% egg::set_panel_size(., width=unit(2.5, "cm"), height=unit(2.5, "cm")), 
        
        ncol=4
    
    )

dev.off()